In [1]:
%%capture
! pip install "ray[tune]"
! pip install mlflow
! pip install datasets
! pip install rdkit
! pip install ray[tune]
! pip install scikit-optimize
! pip install bayesian-optimization==1.4.3
! git clone https://github.com/ChangyunCho/ChemAP

In [2]:
! pip install ray[tune]
! pip install scikit-optimize

In [3]:
def train_valid_test_split(df):

    if isinstance(df ,str):
        df = pd.read_csv(df)
    elif not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a string or a pandas DataFrame")

    test_size = 0.2
    val_size = 0.5
    train_df , temp = train_test_split(df , stratify = df.labels , test_size = test_size , random_state=1234)
    test_df , val_df = train_test_split(temp , stratify = temp.labels , test_size = val_size , random_state=1234)
    #-----------------------------------
    # reset index
    train_df.reset_index(drop=True, inplace=True)
    val_df.reset_index(drop=True, inplace=True)
    test_df.reset_index(drop=True, inplace=True)



    #-----------------------------------
    dataset_train = Dataset.from_pandas(train_df)
    dataset_df = Dataset.from_pandas(val_df)
    dataset_test = Dataset.from_pandas(test_df)


    return train_df , val_df , test_df , dataset_train , dataset_df , dataset_test

In [4]:
def get_canonical_smiles(smiles):
    # Convert input SMILES to molecule object
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        return None  # Invalid SMILES

    # Generate canonical SMILES
    canonical_smiles = Chem.MolToSmiles(mol, isomericSmiles=True, canonical=True)
    return canonical_smiles

In [5]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer , SentenceTransformerTrainer
from sentence_transformers.losses import CachedMultipleNegativesRankingLoss ,ContrastiveLoss, OnlineContrastiveLoss
import os
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
import torch
from transformers import RobertaTokenizer , RobertaForSequenceClassification , TrainingArguments , Trainer,TrainerCallback,AutoModelForSequenceClassification,AutoTokenizer, AutoModelForCausalLM
import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import pandas as pd
import torch
from transformers import AutoModel
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import RDKFingerprint
from rdkit.Avalon.pyAvalonTools import GetAvalonFP
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv("/content/ChemAP/dataset/DrugApp/All_training_feature_vectors.csv")
df["SMILES"] = df["SMILES"].apply(lambda x: get_canonical_smiles(x))
df.rename(columns={"Label":"labels"}, inplace=True)

In [7]:
df= df[["SMILES", "labels"]]

In [38]:
import torch
import numpy as np
import random
import pandas as pd
import mlflow
import mlflow.pytorch
from transformers import TrainingArguments, Trainer
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers.trainer_utils import set_seed
import torch.nn as nn
from datasets import Dataset
from sklearn.utils.class_weight import compute_class_weight
from ray import tune
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.schedulers import HyperBandScheduler
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
# Set manual seeds
set_seed(42)
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


In [9]:
train_df , val_df , test_df , dataset_train , dataset_df , dataset_test = train_valid_test_split(df)

In [10]:
from sklearn.metrics import f1_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix

def compute_metrics(p):
    # Predicted class labels
    preds = p.predictions.argmax(axis=-1)
    labels = p.label_ids

    # Calculate metrics
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    recall = recall_score(labels, preds)

    # Calculate confusion matrix for specificity (TN / (TN + FP))
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0

    # AUC calculation (binary classification assumed)
    auc = roc_auc_score(labels, p.predictions[:, 1]) if len(set(labels)) > 1 else 0.5  # Handle binary case

    return {
        "eval_accuracy": accuracy,
        "eval_f1": f1,
        "eval_recall": recall,
        "eval_specificity": specificity,
        "eval_auc": auc
    }

In [ ]:

# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained(
    "DeepChem/ChemBERTa-77M-MTR"
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Ensure dataset_df is a Hugging Face dataset
if isinstance(dataset_df, pd.DataFrame):
    dataset_df = Dataset.from_pandas(dataset_df)

# Define tokenization function
def tokenize_function(examples):
    return tokenizer(examples['SMILES'], padding="max_length", truncation=True)

tokenized_train = dataset_train.map(tokenize_function)
tokenized_val = dataset_df.map(tokenize_function)
tokenized_test = dataset_test.map(tokenize_function)

# Compute class weights
labels_np = np.array(train_df["labels"])
class_weights = compute_class_weight("balanced", classes=np.unique(labels_np), y=labels_np)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Define custom loss function
class CustomLoss(nn.Module):
    def __init__(self, class_weights):
        super(CustomLoss, self).__init__()
        self.ce_loss = nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, logits, labels):
        ce_loss = self.ce_loss(logits, labels)
        l1_loss = torch.mean(torch.abs(logits))
        return ce_loss + 0.1 * l1_loss

# Define hyperparameter search function
def hyperparameter_search(trial):
    return {
        "learning_rate": tune.loguniform(1e-5, 1e-2),
        "num_train_epochs": tune.choice([5, 10, 15]),
        "per_device_train_batch_size": tune.choice([32, 64, 128]),
        "weight_decay": tune.uniform(0.01, 0.3),
        "warmup_steps": tune.choice([0, 50, 100, 200]),
        "lr_scheduler_type": tune.choice(["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"]),
    }

# Custom Trainer to work with Ray Tune
class TuneTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels", None)
        if labels is not None:
            labels = labels.to(device)
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = CustomLoss(class_weights)
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

    def hyperparameter_search(self, hp_space, **kwargs):
        def _objective(config):
            model = RobertaForSequenceClassification.from_pretrained(
                "DeepChem/ChemBERTa-77M-MTR",
                num_labels=2
            ).to(device)

            args = TrainingArguments(
                output_dir="/tune_checkpoints",
                evaluation_strategy='steps',
                save_strategy='steps',
                learning_rate=config['learning_rate'],
                per_device_train_batch_size=config['per_device_train_batch_size'],
                weight_decay=config['weight_decay'],
                num_train_epochs=config['num_train_epochs'],
                warmup_steps=config['warmup_steps'],
                lr_scheduler_type=config['lr_scheduler_type'],
                logging_dir="/tune_logs",
                logging_steps=1,
                seed=42,
                report_to="mlflow",  # Corrected here
            )

            trainer = TuneTrainer(
                model=model,
                args=args,
                train_dataset=tokenized_train,
                eval_dataset=tokenized_val,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics
            )

            trainer.train()  # Start training

            eval_result = trainer.evaluate()
            mlflow.log_params(config)  # Log hyperparameters

            # Log evaluation metrics
            for key, value in eval_result.items():
                if isinstance(value, (float, int)):  # Only log numeric values
                    mlflow.log_metric(key, value)

            eval_result = trainer.evaluate()

            # Assuming you have computed f1, recall, specificity, AUC, accuracy, etc.
            f1_score = eval_result['eval_f1']
            recall = eval_result['eval_recall']
            accuracy = eval_result['eval_accuracy']
            auc = eval_result['eval_auc']
            specificity = eval_result['eval_specificity']

            # Log hyperparameters and metrics to MLflow
            mlflow.log_params(config)  # Log hyperparameters
            mlflow.log_metric('eval_loss', eval_result['eval_loss'])  # Log evaluation loss
            mlflow.log_metric('eval_f1', f1_score)  # Log F1 score
            mlflow.log_metric('eval_recall', recall)  # Log recall
            mlflow.log_metric('eval_accuracy', accuracy)  # Log accuracy
            mlflow.log_metric('eval_auc', auc)  # Log AUC
            mlflow.log_metric('eval_specificity', specificity)  # Log specificity

            # Report evaluation metrics to Ray Tune
            tune.report(
                eval_loss=eval_result['eval_loss'],  # Log the evaluation loss
                eval_f1=f1_score,                    # Log F1 score
                eval_recall=recall,                  # Log recall
                eval_accuracy=accuracy,             # Log accuracy
                eval_auc=auc,                        # Log AUC
                eval_specificity=specificity         # Log specificity
            )
        scheduler = ASHAScheduler(
            metric="eval_loss",  # Track progress using eval_loss
            mode="min",          # Minimize eval_loss
            max_t=50,            # Maximum number of iterations
            reduction_factor=3,  # Reduce resources by a factor of 3 after each iteration
        )
        hyperopt_search = HyperOptSearch(
        metric="eval_loss",  # Use 'eval_loss' as the optimization metric
        mode="min",          # Minimize the eval_loss
        )
        analysis = tune.run(
            _objective,
            config=hyperparameter_search(None),
            search_alg=hyperopt_search,
            num_samples=10,
            scheduler=scheduler,
            resources_per_trial={"cpu": 2, "gpu": 1},
        )
        return analysis.best_config


# Run hyperparameter tuning
trainer = TuneTrainer(
    model=RobertaForSequenceClassification.from_pretrained(
        "DeepChem/ChemBERTa-77M-MTR",
        num_labels=2
    ).to(device),
    args=TrainingArguments(output_dir="/tmp"),
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

best_hyperparams = trainer.hyperparameter_search(hyperparameter_search)


Map:   0%|          | 0/2497 [00:00<?, ? examples/s]

Map:   0%|          | 0/313 [00:00<?, ? examples/s]

Map:   0%|          | 0/312 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-44-fcdf350ba476>:150: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TuneTrainer.__init__`. Use `processing_class` instead.
  trainer = TuneTrainer(


+-------------------------------------------------------------------+
| Configuration for experiment     _objective_2025-03-24_21-52-06   |
+-------------------------------------------------------------------+
| Search algorithm                 SearchGenerator                  |
| Scheduler                        AsyncHyperBandScheduler          |
| Number of trials                 10                               |
+-------------------------------------------------------------------+

View detailed results here: /root/ray_results/_objective_2025-03-24_21-52-06
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-03-24_21-25-33_062755_13954/artifacts/2025-03-24_21-52-06/_objective_2025-03-24_21-52-06/driver_artifacts`

Trial status: 1 PENDING
Current time: 2025-03-24 21:52:06. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
+---------------------------------------------------------------------------

...

  0%|          | 0/40 [00:00<?, ?it/s]
(_objective pid=21247) 
 15%|█▌        | 6/40 [00:00<00:00, 55.54it/s]
(_objective pid=21247) 
 30%|███       | 12/40 [00:00<00:00, 48.68it/s]
(_objective pid=21247) 
 42%|████▎     | 17/40 [00:00<00:00, 45.98it/s]
(_objective pid=21247) 
 55%|█████▌    | 22/40 [00:00<00:00, 47.02it/s]
(_objective pid=21247) 
 68%|██████▊   | 27/40 [00:00<00:00, 45.25it/s]
(_objective pid=21247) 
 82%|████████▎ | 33/40 [00:00<00:00, 46.95it/s]
(_objective pid=21247) 
                                                 
100%|██████████| 40/40 [00:00<00:00, 46.63it/s]
                                               


(_objective pid=21247) {'eval_accuracy': 0.6900958466453674, 'eval_f1': 0.7313019390581718, 'eval_recall': 0.66, 'eval_specificity': 0.7433628318584071, 'eval_auc': 0.7358849557522122, 'eval_loss': 0.755546510219574, 'eval_runtime': 0.8673, 'eval_samples_per_second': 360.882, 'eval_steps_per_second': 46.119, 'epoch': 11.03}
(_objective pid=21247) {'loss': 0.5157, 'grad_norm': 0.9442911148071289, 'learning_rate': 0.000745695705692058, 'epoch': 11.05}


  0%|          | 0/40 [00:00<?, ?it/s]
(_objective pid=21247) 
 15%|█▌        | 6/40 [00:00<00:00, 57.90it/s]
(_objective pid=21247) 
 30%|███       | 12/40 [00:00<00:00, 50.51it/s]
(_objective pid=21247) 
 45%|████▌     | 18/40 [00:00<00:00, 48.31it/s]
(_objective pid=21247) 
 57%|█████▊    | 23/40 [00:00<00:00, 47.54it/s]
(_objective pid=21247) 
 70%|███████   | 28/40 [00:00<00:00, 46.78it/s]
(_objective pid=21247) 
 82%|████████▎ | 33/40 [00:00<00:00, 45.85it/s]
(_objective pid=21247) 
 95%|█████████▌| 38/40 [00:00<00:00, 45.44it/s]


(_objective pid=21247) {'eval_accuracy': 0.6869009584664537, 'eval_f1': 0.73224043715847, 'eval_recall': 0.67, 'eval_specificity': 0.7168141592920354, 'eval_auc': 0.733716814159292, 'eval_loss': 0.7483080625534058, 'eval_runtime': 0.872, 'eval_samples_per_second': 358.944, 'eval_steps_per_second': 45.871, 'epoch': 11.05}


                                                 
100%|██████████| 40/40 [00:00<00:00, 45.44it/s]
                                               


In [15]:
+! pip install bayesian-optimization